# Example of self-supervised learning using augmentation to do contrastive learning

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Import packages and load data

In [3]:
import sys
sys.path.insert(1, '../src/')

from ete4 import Tree
from mixup import Mixup, PhylogenyDataset, PhylogenyTree
dataset = "ibd200"
target = "type"

data_fp = f'../data/{dataset}/data.tsv.xz'
meta_fp = f'../data/{dataset}/meta.tsv'
target_fp = f'../data/{dataset}/{target}.py'

phylogeny_tree_fp = '../data/WoL2/phylogeny.nwk'
tree = PhylogenyTree.init_from_nwk(phylogeny_tree_fp)
data = PhylogenyDataset.init_from_files(data_fp, meta_fp, target_fp)
tree.prune(data.features)
print("number of leaves in the phylogeny tree after pruning: ", len(list(tree.ete_tree.leaves())))

number of leaves in the phylogeny tree after pruning:  5287


## Get augmented data

Contrastive learning requires a pair of augmented data, and we can using phylomix to generate them by specifying different random seed

In [4]:
import numpy as np

data.one_hot_encode()
mixup = Mixup(data, taxonomy_tree=None, phylogeny_tree=tree)
idx1 = np.arange(len(data.X))
idx2 = idx1[::-1]
out1 = mixup.mixup(len(idx1), method='phylomix', alpha=2, tree='phylogeny', index1=idx1, index2=idx2, contrastive_learning=True, seed=0)
out2 = mixup.mixup(len(idx1), method='phylomix', alpha=2, tree='phylogeny', index1=idx1, index2=idx2, contrastive_learning=True, seed=1)

## Define a simple model as encoder

In [5]:
class EncoderProjectionHead(nn.Module):
    def __init__(self, input_dim, hidden_dim1=1024, hidden_dim2=256, latent_dim=512, output_dim=512):
        super(EncoderProjectionHead, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim // 2)
        self.fc2 = nn.Linear(input_dim // 2, hidden_dim1)
        self.fc3 = nn.Linear(hidden_dim1, output_dim)
        self.encoder = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            self.fc2,
            nn.ReLU(),
            self.fc3
        )
        self.projection_head = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128)
        )

    def forward(self, x):
        z = self.encoder(x)
        z = self.projection_head(z)
        return F.normalize(z, dim=1)  # Normalize the output

## Define the simclr loss function

In [6]:
def info_nce_loss(features, batch_size, device):

    labels = torch.cat([torch.arange(batch_size) for i in range(2)], dim=0).to(device)
    labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
    labels = labels

    features = F.normalize(features, dim=1)

    similarity_matrix = torch.matmul(features, features.T)

    # discard the main diagonal from both: labels and similarities matrix
    mask = torch.eye(labels.shape[0], dtype=torch.bool)
    labels = labels[~mask].view(labels.shape[0], -1)
    similarity_matrix = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1)

    positives = similarity_matrix[labels.bool()].view(labels.shape[0], -1)

    negatives = similarity_matrix[~labels.bool()].view(similarity_matrix.shape[0], -1)

    logits = torch.cat([positives, negatives], dim=1)
    labels = torch.zeros(logits.shape[0], dtype=torch.long)

    logits = logits / 0.05
    return logits.to(device), labels.to(device)

## Make data into data loader

In [7]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
combined_data = np.stack((out1, out2), axis=1)
combined_data = torch.tensor(combined_data, dtype=torch.float32).to(device)
dataloader = DataLoader(combined_data, batch_size=combined_data.shape[0], shuffle=True)

## Train the model

In [8]:
from tqdm import tqdm

input_dim = data.X.shape[-1]
encoder = EncoderProjectionHead(input_dim).to(device)
epochs = 1000
optimizer = optim.Adam(encoder.parameters(), lr=0.001)
batch_size = dataloader.batch_size
criterion = nn.CrossEntropyLoss()


for epoch in range(epochs):
    epoch_loss = 0

    with tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}") as pbar:
        for d in pbar:
            d = d.to(device).view(-1, d.size()[-1])
            optimizer.zero_grad()
            z = encoder(d)
            logits, labels = info_nce_loss(z, batch_size, device)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_description(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item():.6f}")

    avg_loss = epoch_loss / len(dataloader)



Epoch 1000/1000 - Loss: 5.171940: 100%|██████████| 1/1 [00:00<00:00, 57.77it/s]


## Get the embedding

In [9]:
encoder.eval()
with torch.no_grad():
    feature_extractor = encoder.encoder
    embedding = feature_extractor(torch.tensor(data.X).to(device)).cpu().numpy()
embedding.shape

(174, 512)

## We can use the embedding for downstream task

In [22]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score
import warnings

warnings.filterwarnings("ignore")
y = np.argmax(data.y, axis=1)
logistic_cv = LogisticRegressionCV(Cs = 1, max_iter=1000, random_state=0)

y_pred_cv = cross_val_predict(logistic_cv, embedding, y, cv=5, method="predict")

auroc = roc_auc_score(y, y_pred_cv)

accuracy = accuracy_score(y, y_pred_cv)

print(f"Cross-validated Accuracy: {accuracy:.4f}")
print(f"Cross-validated AUROC: {auroc:.4f}")

Cross-validated Accuracy: 0.5345
Cross-validated AUROC: 0.5072
